# LLMRouterChain

In [1]:
import openai
import os 
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [2]:
os.environ['OPENAI_API_KEY'] = ''

In [3]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
llm = ChatOpenAI()

In [5]:
hr_template="""You are a very smart human resources person. \
You are great at answering questions about human resources in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


In [6]:
law_template="""You are a very smart lawyer. \
You are great at answering questions about the law in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

In [7]:
prompt_infos = [
{
    'name':'human resources',
    'description':'Answer human resources questions',
    'template':hr_template},

{
    'name':'law',
    'description':'Answers legal questions',
    'template':law_template}
]

In [8]:
destination_chains ={}

In [9]:
for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info['template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm,prompt=prompt)
    destination_chains[name] = chain

In [10]:
default_prompt = ChatPromptTemplate.from_template('{input}')

default_chain = LLMChain(llm=llm,prompt=default_prompt)

In [11]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:

destinations=[
f"{p['name']}: {p['description']}" for p in prompt_infos
]


In [13]:
destinations

['human resources: Answer human resources questions',
 'law: Answers legal questions']

In [14]:
destination_str = "\n".join(destinations)

print(destination_str)

human resources: Answer human resources questions
law: Answers legal questions


In [15]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)

router_template

'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nhuman resources: Answer human resources questions\nlaw: Answers legal questions\n\

In [17]:
router_prompt = PromptTemplate(template=router_template,
input_variables=['input'],
output_parser=RouterOutputParser())

router_chain = LLMRouterChain.from_llm(llm,router_prompt)

chain = MultiPromptChain(router_chain=router_chain,
destination_chains=destination_chains,
default_chain=default_chain,verbose=True)

chain.run("How do intellectual property laws address the use of AI-generated content?")



> Entering new MultiPromptChain chain...
law: {'input': 'How do intellectual property laws address the use of AI-generated content?'}
> Finished chain.


"Thank you for your kind words. Intellectual property laws are still evolving to address the use of AI-generated content. Generally, the creator of the AI-generated content would own the copyright to it. However, if the AI-generated content is created by an AI without human involvement, the question of ownership becomes more complex. Some jurisdictions may consider the AI itself as the author, while others might attribute the creation to the person or organization that owns or operates the AI. To navigate this issue, countries may need to adapt their laws and establish clear guidelines regarding ownership and rights in AI-generated content. It's important to consult with legal professionals who specialize in intellectual property law to get the most accurate and up-to-date advice."

In [18]:
chain.run("How do you handle conflict between team members?")



> Entering new MultiPromptChain chain...
human resources: {'input': 'How do you handle conflict between team members?'}
> Finished chain.


"Handling conflict between team members is a crucial part of being an effective human resources professional. Here are some steps you can take to address conflicts:\n\n1. Encourage open communication: Encourage the team members involved to communicate directly with each other, emphasizing active listening and respectful dialogue.\n\n2. Create a safe space for discussion: Provide a neutral and confidential environment where team members can express their concerns and feelings without fear of judgment or retaliation.\n\n3. Act as a mediator: If necessary, step in as a neutral third party to facilitate a discussion between the conflicting parties, helping them understand each other's perspectives.\n\n4. Identify the root cause: Explore the underlying reasons for the conflict, as it could be due to differences in work styles, miscommunication, or conflicting goals. Understanding the root cause helps in finding effective resolutions.\n\n5. Encourage empathy and understanding: Promote empath